In [1]:
import pandas as pd
import numpy as np
import os

from pathlib import Path

In [2]:
VERIFIERS = [
    'neurify',
    'verinet',
    'nnenum',
    'marabou',
    'planet',
    'eran',
    'bab',
    'reluplex',
    'mipverify',
]

def read_log_dnnp(params):
    result_file = open(params['log_path'], "r")
    params['status'] = '???'
    params['error_type'] = None
    params['time'] = 0
    tool_executed = False
    for line in result_file:
        if("EXECUTING:" in line or "Starting Falsifier" in line):
            tool_executed = True
        if("result: " in line):
            params['status'] = line.split('result: ')[1].split('\n')[0]
            if "error" in params['status'].lower():
                params['error_type'] = params['status']
                params['status'] = "error"
        if("  time: " in line):
            params['time'] = float(line.split("time: ")[1].split('\n')[0])
        elif("  total time: " in line):
            params['time'] = float(line.split("total time: ")[1].split('\n')[0])
        if("(resmonitor) Out of Memory:" in line):
            params['status'] = "error"
            params['error_type'] = "Memory excedeed"
        if("(resmonitor) Timeout:" in line):
            params['status'] = "timeout"
            time = float(line.split("(resmonitor) Timeout:")[-1].split('\n')[0].split('>')[-1])
            params['time'] = time
    if params['status'] == "timeout" and tool_executed is False:
        params['status'] = "timeout - Tool never executed"

In [3]:
result_path = Path("./results/")
df = None
for tool in os.listdir(result_path):
    log_path = result_path / tool / "logs"
    for log in os.listdir(log_path):
        prop_id = log.split('.txt')[0]
        prop_type = prop_id.split('_')[0]
        epsilon = None
        if prop_type == "pr":
            epsilon = prop_id.split('_')[-1]
        ce_name = prop_id + ".npy"
        ce_path = log_path.parent / "ces" / ce_name
        params = {
            'prop_id': prop_id,
            'prop_type': prop_type,
            'epsilon': epsilon,
            'tool': tool,
            'tool_type': 'verifier' if tool in VERIFIERS else 'falsifier',
            'log_path': str(log_path / log),
            'ce_path': ce_path if os.path.exists(ce_path) else None
        }
        read_log_dnnp(params)
        
        if df is None:
            df = pd.DataFrame(columns=list(params.keys()))
        
        row = list(params.values())
        idx = len(df)
        df.loc[idx] = row

In [6]:
pb = df[df['prop_type'] == "pb"]
pb.loc[(pb["status"] == "error"), "status"] = "err"
pb.loc[(pb["status"] == "timeout"), "status"] = "unk"
pb.loc[(pb["status"] == "unknown"), "status"] = "unk"

/home/felipe/miniconda3/envs/RL/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
q = pb.groupby(['tool', 'prop_id', 'status']).agg({"time":["first"]}).droplevel(0,axis=1).reset_index()
q['combine'] = q[['status','first']].apply(lambda x: x[0] + " (" + str(round(x[1],2)) + ")", 1)
q.drop(['status', 'first'], axis=1, inplace=True)

q = pd.pivot(q, values=['combine'], index=['prop_id'], columns=['tool'])
q.droplevel(0,axis=1).reset_index()

tool,prop_id,cleverhans.BIM,cleverhans.FGM,cleverhans.PGD,marabou,neurify,nnenum,verinet
0,pb_pole_angle_neg,unk (0.57),unk (0.24),unk (600.0),err (2.69),sat (0.48),sat (13.94),unsat (0.06)
1,pb_pole_angle_pos,unk (0.47),unk (0.26),unk (600.0),err (0.65),sat (0.41),sat (3.56),unsat (0.06)
2,pb_pole_angle_vel_neg,unk (0.46),sat (0.26),sat (0.64),err (0.7),sat (0.43),sat (3.29),err (9.54)
3,pb_pole_angle_vel_pos,sat (0.46),unk (0.26),sat (0.43),err (0.62),unk (600.0),sat (3.34),err (2.66)
4,pb_position,sat (0.43),unk (0.25),sat (0.43),err (0.66),sat (0.42),sat (3.13),err (2.38)


In [8]:
pb.groupby('tool').sum()

,time
tool,
cleverhans.BIM,2.3738
cleverhans.FGM,1.2749
cleverhans.PGD,1201.5035
marabou,5.3216
neurify,601.7312
nnenum,27.2739
verinet,14.7171


In [13]:
pr = df[df['prop_type'] == "pr"]
pr.loc[(pr["status"] == "error"), "status"] = "err"
pr.loc[(pr["status"] == "timeout"), "status"] = "unk"
pr.loc[(pr["status"] == "unknown"), "status"] = "unk"

/home/felipe/miniconda3/envs/RL/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [14]:
pr['prop_id'] = pr['prop_id'].apply(lambda x: '_'.join(x.split('_')[:-1]))

/home/felipe/miniconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
q = pr.groupby(['tool','prop_id','status']).epsilon.count().reset_index()
# pd.pivot(q, values="prop_id", index="tool", columns="status")
q['combine'] = q[['status','epsilon']].apply(lambda x: str(x[1]) + " " + x[0], 1)
q.drop(['status','epsilon'], axis=1, inplace=True)


q = pd.pivot(q, values=['combine'], index=['prop_id'], columns=['tool'])
q = q.droplevel(0,axis=1).reset_index()
q

tool,prop_id,cleverhans.BIM,cleverhans.FGM,cleverhans.PGD,marabou,neurify,nnenum,verinet
0,pr_all_percentage,5 unk,5 unk,5 unk,5 err,5 unsat,5 unsat,5 err
1,pr_cart_vel_percentage,5 unk,5 unk,5 unk,5 err,5 unsat,5 unsat,5 err
2,pr_pole_angle_percentage,5 unk,5 unk,5 unk,5 err,5 unsat,5 unsat,5 err
3,pr_pole_angle_vel_percentage,5 unk,5 unk,5 unk,5 err,5 unsat,5 unsat,5 err
4,pr_position_percentage,5 unk,5 unk,5 unk,5 err,5 unsat,5 unsat,5 err


In [12]:
pr.groupby('tool').sum()

,time
tool,
cleverhans.BIM,12.2859
cleverhans.FGM,6.8607
cleverhans.PGD,15000.0000
marabou,15.3796
neurify,12.4083
nnenum,50.8303
verinet,58.0892
